In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')
import plotutils

In [3]:
import fitbit
import os
import getpass
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show, output_file, save
from bokeh.models import Range1d
from bokeh.layouts import gridplot

import pickle
output_notebook()

Loading BokehJS ...

## Register your application

To get client ID and secret, register a personal application at www.dev.fitbit.com. See `img/register_app.png` for settings I used.

IMPORTANT: The callback url in `src/generate_tokens.py` and in the application settings must match exactly. Even a missing trailing forward slash `/` will result connection error!

In [4]:
os.environ['client_id'] = '22BC6L'
os.environ['client_secret'] = 'e8348b3b97bba4db4fb142928cedfc31'

Modify the shell command below to use the fitbit python environment and run the `gengerate_tokens.py` script. This script takes in the client ID and secret as input and produces the access token files in `../data/`.

In [5]:
!/Users/hasannagib/opt/anaconda3/envs/fitbit/bin/python ../src/generate_tokens.py $client_id $client_secret

[06/Feb/2020:17:43:37] ENGINE Listening for SIGTERM.
[06/Feb/2020:17:43:37] ENGINE Listening for SIGHUP.
[06/Feb/2020:17:43:37] ENGINE Listening for SIGUSR1.
[06/Feb/2020:17:43:37] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[06/Feb/2020:17:43:37] ENGINE Started monitor thread 'Autoreloader'.
[06/Feb/2020:17:43:37] ENGINE Serving on http://127.0.0.1:8080
[06/Feb/2020:17:43:37] ENGINE Bus STARTED
127.0.0.1 - - [06/Feb/2020:17:43:45] "GET /?code=3e8367bf00096eedcff87a0d7a25ae91729dd4cf&state=c2qOGys7utOccqpEw294qSw778xDmX HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"
127.0.0.1 - - [06/Feb/2020:17:43:45] "GET /favicon.ico HTTP/1.1" 304 - "http://127.0.0.1:8080/?code=3e8367bf00096eedcff87a0d7a25ae91729dd4cf&state=c2qOGys7utOccqpEw294qSw778xDmX" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945

## Read tokens

In [6]:
with open('../data/access_token.txt','r') as f:
    os.environ['fitbit_access_token'] = f.read()

with open('../data/refresh_token.txt','r') as f:
    os.environ['fitbit_refresh_token'] = f.read()

## Login to Fitbit

In [7]:
auth_client = fitbit.Fitbit(
    os.environ['client_id'], 
    os.environ['client_secret'],
    os.environ['fitbit_access_token'],
    os.environ['fitbit_refresh_token']
)

## Query sleep data for a date

The `.get_sleep()` method gets sleep data for a given given date. 

In [9]:
sleep = auth_client.get_sleep(pd.to_datetime('2019-05-01'))['summary']['stages']
plotutils.sleep_summary_plot(sleep)

## Query sleep data over a date range

Fitbit limits user to 120 API calls per hour. So `.get_sleep()` wouldn't be ideal for getting sleep data over a longer period. Instead we can use the `.time_series()` which is an interface for [this](https://dev.fitbit.com/build/reference/web-api/sleep/#get-sleep-logs-by-date-range) date range API. 

This, however, only allows 100 data points to be extracted at a time. So in order to get more than 100 days worth of sleep data, we would have to split up our date range in chunks that are less than or equal to 100. 

In [ ]:
# Split up date range into smaller chunks
dates = pd.date_range('2020-01-27',pd.datetime.today())
date_splits = np.array_split(dates, np.ceil(len(dates)/100))
date_splits_endpoints = [(date_split[0], date_split[-1]) for date_split in date_splits]

date_splits_endpoints

In [ ]:
# Pull sleep data for each split
sleep = []
for endpoint in date_splits_endpoints:
    sleep += auth_client.time_series(
        resource='sleep',
        base_date=endpoint[0],
        end_date=endpoint[-1]
    )['sleep']

Here is what the API call output looks like: 

```json
{
    "sleep": [
        {
            "dateOfSleep": "2017-04-02",
            "duration": <value in milliseconds>,
            "efficiency": <value>,
            "isMainSleep": <true|false>,
            "levels": {
                "summary": {
                    "deep": {
                        "count": <value>,
                        "minutes": <value>,
                        "thirtyDayAvgMinutes": <value>
                    },
                    "light": {
                        "count": <value>,
                        "minutes": <value>,
                        "thirtyDayAvgMinutes": <value>
                    },
                    "rem": {
                        "count": <value>,
                        "minutes": <value>,
                        "thirtyDayAvgMinutes": <value>
                    },
                    "wake": {
                        "count": <value>,
                        "minutes": <value>,
                        "thirtyDayAvgMinutes": <value>
                    }
                },
                "data": [
                    {
                        "datetime": "2017-04-01T23:58:30.000",
                        "level": "wake",
                        "seconds": <value>
                    },
                    {
                        "datetime": "2017-04-02T00:16:30.000",
                        "level": "light",
                        "seconds": <value>
                    },
                    <...>
                ],
                "shortData": [
                    {
                        "datetime": "2017-04-02T05:58:30.000",
                        "level": "wake",
                        "seconds": <value>
                    },
                    <...>
                ]
            },
            "logId": <value>,
            "minutesAfterWakeup": <value>,
            "minutesAsleep": <value>,
            "minutesAwake": <value>,
            "minutesToFallAsleep": <value>, // this is generally 0 for autosleep created sleep logs
            "startTime": "2017-04-01T23:58:30.000",
            "timeInBed": <value in minutes>,
            "type": "stages"
        },
        <...>
    ]
}

```

Let's parse some of the data from this and store in a dataframe

In [ ]:
# Parse sleep data range API output 
items = ['timeInBed', 'minutesAsleep', 'dateOfSleep', 'minutesToFallAsleep', 'minutesAwake']
sleep_data = [{item:s[item] for item in items} for s in sleep]

In [ ]:
# Create dataframe 
df = pd.DataFrame(sleep_data)
df['dateOfSleep'] = pd.to_datetime(df['dateOfSleep'])
df = df.sort_values('dateOfSleep').reset_index()

## `matplotlib`

In [ ]:
df_plot = df.query('minutesAsleep > 0').groupby(pd.Grouper(key='dateOfSleep', freq='w')).mean()

df_plot['timeInBed'].plot(
    figsize=(12,4),
    grid=True,
    style='o-',
    alpha=0.5
);

plt.hlines(450, df_plot.index.min(), df_plot.index.max())
plt.legend();

## `bokeh`

In [ ]:
df_plot['Goal'] = 8*60
plotutils.plot_ts(
    df_plot, 
    date_col='dateOfSleep',
    ys=['minutesAsleep', 'Goal'], 
    hover_vars=['minutesAsleep', 'index'],
    styles=['o-', '-'],
    plot_height=300,
    plot_width=800
)

Unfortunately this time series data is not quite accurate... so we have to resort back to the `get_sleep()` method. I will pull `100` days of sleep data at a time and save in the `../data/` directory.

In [ ]:
dates = pd.date_range('2020-01-27','2020-02-02')

In [ ]:
sleep_20180902_20181222 = [auth_client.get_sleep(date) for date in dates]

In [ ]:
with open('../data/sleep_20200127_20200202.pkl', 'wb') as file:
    pickle.dump(sleep_20200127_20200202, file)

In [8]:
sleep_list = []

with open('../data/sleep_20180902_20181222.pkl', 'rb') as handle:
    sleep_list += pickle.load(handle)

with open('../data/sleep_20181223_20190401.pkl', 'rb') as handle:
    sleep_list += pickle.load(handle)

with open('../data/sleep_20190402_20190710.pkl', 'rb') as handle:
    sleep_list += pickle.load(handle)

with open('../data/sleep_20190711_20191018.pkl', 'rb') as handle:
    sleep_list += pickle.load(handle)
    
with open('../data/sleep_20191019_20200126.pkl', 'rb') as handle:
    sleep_list += pickle.load(handle)
    
with open('../data/sleep_20200127_20200202.pkl', 'rb') as handle:
    sleep_list += pickle.load(handle)

In [9]:
df_sleep_details = pd.concat([pd.DataFrame(sleep['sleep']) for sleep in sleep_list])
df_sleep_details['dateOfSleep'] = pd.to_datetime(df_sleep_details['dateOfSleep'])
df_sleep_details = df_sleep_details.set_index('dateOfSleep')

In [10]:
df_sleep_summary_list = []
for sleep in sleep_list:
    try:
        df_sleep_summary_list.append(
            pd.DataFrame(
                sleep['summary']['stages'], index=[pd.to_datetime(sleep['sleep'][0]['dateOfSleep'])]
            )
        )
    except KeyError:
        pass
                                                                                 
                                                                                 
df_sleep_summary = pd.concat(df_sleep_summary_list)
df_sleep_summary.index.name = 'dayOfSleep'

In [11]:
df_sleep = df_sleep_details.join(df_sleep_summary)
df_sleep.index.name = 'dayOfSleep'

In [12]:
df_sleep['timeAsleep'] = df_sleep['deep'] + df_sleep['light'] + df_sleep['rem']
df_sleep['timeAsleepHours'] = df_sleep['timeAsleep']//60
df_sleep['timeAsleepMinutes'] = df_sleep['timeAsleep']%60
df_sleep['sleepEfficiency'] = 1 - (df_sleep['wake']/df_sleep['timeAsleep'])
df_sleep['timeInBed']= df_sleep['timeAsleep'] + df_sleep['minutesAwake'] 

In [13]:
df_sleep.head()

,awakeCount,awakeDuration,awakeningsCount,duration,efficiency,endTime,isMainSleep,logId,minuteData,minutesAfterWakeup,...,startTime,timeInBed,deep,light,rem,wake,timeAsleep,timeAsleepHours,timeAsleepMinutes,sleepEfficiency
dayOfSleep,,,,,,,,,,,,,,,,,,,,,
2018-09-02,9,12,32,41340000,92,2018-09-02T08:45:41.000,True,19359506461,"[{'dateTime': '21:16:00', 'value': '2'}, {'dat...",0,...,2018-09-01T21:16:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-03,2,7,9,24600000,97,2018-09-03T08:06:10.000,True,19359065401,"[{'dateTime': '01:16:00', 'value': '3'}, {'dat...",0,...,2018-09-03T01:16:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-04,2,7,22,33540000,93,2018-09-04T07:56:00.000,True,19401471074,"[{'dateTime': '22:37:00', 'value': '1'}, {'dat...",0,...,2018-09-03T22:37:00.000,510.0,91.0,260.0,123.0,85.0,474.0,7.0,54.0,0.820675
2018-09-05,0,0,10,21180000,94,2018-09-05T08:52:30.000,True,19415972110,"[{'dateTime': '02:59:30', 'value': '2'}, {'dat...",0,...,2018-09-05T02:59:30.000,324.0,78.0,158.0,66.0,51.0,302.0,5.0,2.0,0.831126
2018-09-06,3,14,26,27300000,91,2018-09-06T08:43:30.000,True,19424185340,"[{'dateTime': '01:08:00', 'value': '1'}, {'dat...",0,...,2018-09-06T01:08:00.000,435.0,79.0,232.0,83.0,61.0,394.0,6.0,34.0,0.845178


In [14]:
df_sleep['startHour'] = pd.to_datetime(df_sleep['startTime']).dt.hour + pd.to_datetime(df_sleep['startTime']).dt.minute/60
df_sleep['startHour'] = df_sleep['startHour'].apply(lambda x: x+24 if (x<15) else x)
df_sleep['endHour'] = pd.to_datetime(df_sleep['endTime']).dt.hour + pd.to_datetime(df_sleep['endTime']).dt.minute/60

In [22]:
df_sleep['8hr00min'] = 8*60
df_sleep['7hr30min'] = 7.5*60

p_duration = plotutils.plot_ts(
    df_sleep.groupby(pd.Grouper(freq='W')).mean(), 
    date_col='dayOfSleep',
    ys=['timeAsleep', 'timeInBed', '8hr00min', '7hr30min'], 
    hover_vars=['timeAsleep'],
    styles=['|','|','--', '--'],
    plot_height=280,
    plot_width=600,
    title='Sleep Duration - Weekly Average',
    xlabel='Date',
    ylabel='Minute',
    legend_location='bottom_left'
)

p_duration.y_range=Range1d(120, 600)

show(p_duration)

In [17]:
p_eff = plotutils.plot_ts(
    df_sleep.groupby(pd.Grouper(freq='w')).mean(), 
    date_col='dayOfSleep',
    ys=['sleepEfficiency'], 
    hover_vars=['timeAsleep'],
    styles=['|'],
    plot_height=400,
    plot_width=900,
    title='Sleep Efficiency',
    xlabel='Date',
    ylabel='Minute'
)
p_eff.y_range=Range1d(0.5, 1)



In [23]:
p_hours = plotutils.plot_ts(
    df_sleep.groupby(pd.Grouper(freq='d')).mean(), 
    date_col='dayOfSleep',
    ys=['startHour', 'endHour'], 
    hover_vars=['timeAsleep'],
    styles=['o-'],
    plot_height=280,
    plot_width=600,
    title='Daily Sleep Schedule',
    xlabel='Date',
    ylabel='24 Hour', 
    x_range=p_duration.x_range,
    legend_location='bottom_left'
)
p_hours.y_range=Range1d(0, 35)

show(p_hours)

In [24]:
sleep_dashboard = gridplot([[p_duration], [p_hours]])
show(sleep_dashboard)

In [20]:
output_file("../data/sleep_dashboard.html")
save(sleep_dashboard)

'/Users/hasannagib/Desktop/projects/fitbit-analytics/data/sleep_dashboard.html'

In [22]:
df_sleep.to_csv('../data/sleep_data.csv')